## Pull in the data.

In [467]:
import yfinance as yf
import pandas as pd
from numpy import linalg
import numpy as np
from IPython.display import display, HTML

# define the ticker string.
tickerSymbol = ['MSFT', 'A', 'V', 'F', 'X', 'AA', 'JPM', 'XOM', 'BAC', 'GE', 'ABB', 'WMT', 'HPQ']

# Pull in Price data from yahoo finance
ticker = 'ACB'
tickerData = yf.Ticker(ticker)
tickerData = tickerData.history(period='1d', start='2019-01-01', end='2020-01-25')    
tickerData['TickString'] = ticker
tickerData

for tick in tickerSymbol:
    tickerDataRetrieve = yf.Ticker(tick)
    tickerDf = tickerDataRetrieve.history(period='1d', start='2019-01-01', end='2020-01-25')    
    tickerDf['TickString'] = tick
    tickerData = tickerData.append(tickerDf)

tickerData = tickerData.reset_index()
tickerData = tickerData[['Date', 'Close', 'TickString']]


# Convert to Pct_Change
tickerData = tickerData.set_index(['Date', 'TickString'])
tickerData = tickerData.groupby(level="TickString").pct_change(1)

tickerData.columns = ['Pct_Change']
tickerData = tickerData.reset_index()

# Convert from long to wide format
tickerD = pd.pivot_table(tickerData, index=["Date"], columns=['TickString'], values=["Pct_Change"])

# Remove any NaNs
tickerD = tickerD.dropna(axis=0)
tickerD.reset_index(drop=True, inplace=True)

# Demean the columns
tickerD = tickerD - tickerD.mean()

## Spot check the security price changes.

In [468]:
tickerD.iloc[0:3,1:9]

Pct_Change                                                    \
TickString         AA       ABB       ACB       BAC         F        GE   
0            0.001634 -0.021109 -0.016539 -0.017076 -0.015982 -0.000583   
1            0.081664  0.035108  0.018110  0.040055  0.038180  0.018895   
2            0.004457 -0.004466  0.010208 -0.002106  0.024632  0.060457   

                                
TickString       HPQ       JPM  
0          -0.034284 -0.015488  
1           0.039103  0.035574  
2           0.010123 -0.000681

In [469]:
# Convert to a matrix.
matrixD = tickerD.to_numpy()

# Calculate the Covariance Matrix.
Sigma = (matrixD.transpose() @ matrixD)
Sigma = Sigma * (1/len(matrixD))

# Calculate the Eigenvalues and Eigenvectors 
L, Q = linalg.eig(Sigma)

## Spot check the eigenvalues.

In [470]:
# Cumulative percentage of variance explained by eigenvectors.
(L / L.sum()).cumsum()

array([0.37982705, 0.66046231, 0.74672299, 0.80784509, 0.85938239,
       0.89460973, 0.92288691, 0.9440816 , 0.96063666, 0.96386499,
       0.96929477, 0.97824324, 0.9896163 , 1.        ])

## Run the regressions against the principal components

In [471]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

# print("Number of Securities: " + str(tickerD.shape[1]))
# print("Number of Days: " + str(tickerD.shape[0]))

# Create the 'Eigenvector Portfolios'
eigenVectorP1 = matrixD @ Q[1] # Principal Component 1
eigenVectorP2 = matrixD @ Q[2]
eigenVectorP3 = matrixD @ Q[3]
eigenVectorP4 = matrixD @ Q[4]
eigenVectorP5 = matrixD @ Q[5]
eigenVectorP6 = matrixD @ Q[6]
eigenVectorP7 = matrixD @ Q[7]
eigenVectorP8 = matrixD @ Q[8]
eigenVectorP9 = matrixD @ Q[9]
eigenVectorP10 = matrixD @ Q[10]

independent = np.array([eigenVectorP1, eigenVectorP2]).transpose()
dependent = np.array(matrixD.transpose()[2])
# regressionData = regressionData.to_numpy()

# Regression Residuals for Microsoft
fittedmodel = model.fit(independent, dependent)
dependentNew = dependent.reshape(-1,1)
MSE = ((dependent - fittedmodel.predict(independent)) ** 2).sum()
print("Mean Squared Error using 2 principal components: " + str(MSE))

independent = np.array([eigenVectorP1, eigenVectorP2, eigenVectorP3, eigenVectorP4, eigenVectorP5]).transpose()
dependent = np.array(matrixD.transpose()[2])
# regressionData = regressionData.to_numpy()

# Regression Residuals for Microsoft
fittedmodel = model.fit(independent, dependent)
dependentNew = dependent.reshape(-1,1)
MSE = ((dependent - fittedmodel.predict(independent)) ** 2).sum()
print("Mean Squared Error using 5 principal components: " + str(MSE))

independent = np.array([eigenVectorP1, eigenVectorP2, eigenVectorP3, eigenVectorP4, eigenVectorP5, eigenVectorP6,
                        eigenVectorP7, eigenVectorP8]).transpose()
dependent = np.array(matrixD.transpose()[2])
# regressionData = regressionData.to_numpy()

# Regression Residuals for Microsoft
fittedmodel = model.fit(independent, dependent)
dependentNew = dependent.reshape(-1,1)
MSE = ((dependent - fittedmodel.predict(independent)) ** 2).sum()
print("Mean Squared Error using 8 principal components: " + str(MSE))


Mean Squared Error using 2 principal components: 0.02887802959156023
Mean Squared Error using 5 principal components: 0.026816924847973406
Mean Squared Error using 8 principal components: 0.01689229995995377


## Pulling data from Quandl.

In [472]:
import quandl
quandl.ApiConfig.api_key = 'tsyyEstQNW5wNoqQM4bC'


mydata = quandl.get(['CHRIS/ICE_T1','CHRIS/CME_HG10', 'CHRIS/CME_NN10', 'CHRIS/EUREX_FMEM2'], start_date="2019-01-01", end_date="2020-01-25")
mydata2 = quandl.get(['CHRIS/CME_TN1', 'CHRIS/CME_GC10', 'CHRIS/LIFFE_EMA2'], start_date="2019-01-01", end_date="2020-01-25")


### Factor - time series
#### - LIFFE_EMA2 - Corn Futures
#### - ICE_T1 - WTI Futures
#### - CME_TN1 - U.S 10 year Futures
#### - CME_HG10 - Copper Futures
#### - CME_GC10 - Gold Futures
#### - CME_NN10 - Natural Gas
#### - EUREE_FMEM2 - MSCI Emerging Market Index

In [473]:
# Use regex to filter only 'Settle' columns  
import re
mydata1 = mydata
mydata1 = mydata1.filter(regex=("Settle"))
mydata2 = mydata2.filter(regex=('Settle'))

factorData = mydata1.join(mydata2)

In [474]:
# Clean up the column names.
from nltk import flatten
columnNames = factorData.columns.tolist()

columnNamesNew = []
for col in columnNames:
    x = re.findall(r'/(.*?)[\s]',col)
    columnNamesNew.append(x)
    
columnNamesNew = flatten(columnNamesNew)
factorData.columns = columnNamesNew

In [475]:
# Clean up NaNs by carrying forward chronologically.
factorData = factorData.fillna(method="ffill")

In [476]:
# Calculate the Percent Change
factorData = factorData.pct_change(1)
factorData = factorData.dropna(axis=0)

In [477]:
# Guarantee that factor data and security data are the same - quick and dirty 
# Why are there additional dates in the factor returns?
factorData = factorData[factorData.index.isin(tickerData['Date'])]

In [478]:
# Demean the factors and calculate the factor covariance matrix (Omega)

# Demean the factor data
factorData = factorData - factorData.mean()

# Calculate the factor covariance matrix
factorMatrix = factorData.to_numpy()

# Calculate the sample factor covariance matrix
Omega = (factorMatrix.transpose() @ factorMatrix)
Omega = Omega * (1/len(factorMatrix))

## Sense check the Sample Covariance Matrix

In [479]:
Cov = pd.DataFrame(Omega)
Cov.columns = factorData.columns
Cov
# This seems really low - Correlation matrix!

,ICE_T1,CME_HG10,CME_NN10,EUREX_FMEM2,CME_TN1,CME_GC10,LIFFE_EMA2
0,0.000453,0.000042,0.000055,0.000049,-1.574034e-05,0.000005,4.800389e-06
1,0.000042,0.000099,0.000014,0.000050,-1.188578e-05,0.000005,3.723612e-06
2,0.000055,0.000014,0.000158,0.000005,-8.622707e-06,0.000005,-3.493055e-06
3,0.000049,0.000050,0.000005,0.000092,-1.201895e-05,0.000002,3.209419e-06
4,-0.000016,-0.000012,-0.000009,-0.000012,1.461904e-05,0.000005,-4.747292e-07
5,0.000005,0.000005,0.000005,0.000002,5.051107e-06,0.000043,4.952620e-06
6,0.000005,0.000004,-0.000003,0.000003,-4.747292e-07,0.000005,3.794239e-05


## Sense check the Sample Correlation Matrix

In [480]:
# Convert the covariance matrix into a correlation matrix
L, Q = linalg.eig(Cov)
L = np.diag(L)
CorrMatrix = np.linalg.inv(L) @ Cov @ np.linalg.inv(L)

In [481]:
CorrMatrix = pd.DataFrame(CorrMatrix)
CorrMatrix.columns = factorData.columns
CorrMatrix

,ICE_T1,CME_HG10,CME_NN10,EUREX_FMEM2,CME_TN1,CME_GC10,LIFFE_EMA2
0,1990.318726,589.570338,864.030667,9477.854019,-944.358792,231.083461,211.922736
1,589.570338,4401.340833,713.215996,30294.712813,-2273.379635,781.366792,524.067357
2,864.030667,713.215996,8914.798282,3197.014449,-1853.465504,881.490476,-552.490269
3,9477.854019,30294.712813,3197.014449,768300.386743,-31465.734651,4838.396768,6182.671445
4,-944.358792,-2273.379635,-1853.465504,-31465.734651,11975.704397,3304.673604,-286.158339
5,231.083461,781.366792,881.490476,4838.396768,3304.673604,22462.494855,2384.269105
6,211.922736,524.067357,-552.490269,6182.671445,-286.158339,2384.269105,16829.198995


In [482]:
# Calculate the Betas and Residuals
print("The factor matrix dimensions are: " + str(factorMatrix.shape))
print("The security matrix dimensions are: " + str(matrixD.shape))

The factor matrix dimensions are: (267, 7)
The security matrix dimensions are: (267, 14)


In [483]:
BetaMatrix = []
ResidualMatrix = []
for col in np.arange(matrixD.shape[1]):
    # linModel = model.fit(factorMatrix, matrixD[:,col])
    BetaMatrix.append(model.fit(factorMatrix, matrixD[:,col]).coef_)
    betaCoef = model.fit(factorMatrix, matrixD[:,0]).coef_
    intercept = model.fit(factorMatrix, matrixD[:,0]).intercept_
    # Calculate the residuals
    # matrixD[:,0] = factorMatrix @ betaCoef + intercept + residual
    ResidualMatrix.append(matrixD[:,col] - (factorMatrix @ betaCoef) + intercept)
    
BetaMatrix = np.array(BetaMatrix)
ResidualMatrix = np.array(ResidualMatrix)

In [484]:
# Calculate the factor derived Covariance Matrix
FactorDerivedCovarianceMatrix = BetaMatrix @ Cov @ BetaMatrix.transpose() + ResidualMatrix @ ResidualMatrix.transpose()
FactorDerivedCovarianceMatrix

# Discuss the residual matrix (pg 200-205)
# Is this the correct calculation - what does it mean?
# What should be make of the the fact that the matrix is diagonal
# What do we mean in terms of model specification - strict factor model: expost / exante 
# [We demean but can't guarantee other properties]
# EM Algorithm

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.044646,0.003598,0.004550,0.000065,0.008602,0.007144,0.006218,0.006081,0.007591,0.011418,0.011129,0.006990,0.002232,0.004607
1,0.003598,0.134321,0.008343,0.012518,0.018134,0.019806,0.038427,0.010942,0.011187,-0.000901,-0.008612,-0.013192,0.094465,0.012406
2,0.004550,0.008343,0.030032,0.012301,0.010075,0.011079,0.012988,0.017774,0.010710,0.009001,0.005536,0.003349,0.012118,0.007964
3,0.000065,0.012518,0.012301,0.566003,0.019938,0.004783,0.040779,0.016665,0.014179,0.016563,0.014738,0.007746,-0.005783,0.018781
4,0.008602,0.018134,0.010075,0.019938,0.034685,0.008794,0.016894,0.012437,0.021401,0.005556,0.002692,0.002520,0.022118,0.008053
5,0.007144,0.019806,0.011079,0.004783,0.008794,0.062247,0.024545,0.008483,0.010936,0.003551,0.004585,-0.000093,0.021554,0.008304
6,0.006218,0.038427,0.012988,0.040779,0.016894,0.024545,0.151645,0.030673,0.012406,0.006662,-0.000475,-0.004989,0.047600,0.015366
7,0.006081,0.010942,0.017774,0.016665,0.012437,0.008483,0.030673,0.084473,0.010876,0.013713,0.008453,0.000952,0.028043,0.005646
8,0.007591,0.011187,0.010710,0.014179,0.021401,0.010936,0.012406,0.010876,0.022998,0.005601,0.004549,0.005639,0.012657,0.008249
9,0.011418,-0.000901,0.009001,0.016563,0.005556,0.003551,0.006662,0.013713,0.005601,0.029991,0.018706,0.008401,-0.003442,0.005701


## Orthogonalize the factors through regression

In [485]:
print("Factor data before: ")
print(factorData.head(5))

for col in np.arange(1,factorData.shape[1]):
        # Calculate the residuals
        betaCoef = model.fit(factorData.iloc[:,0:col], factorData.iloc[:,col]).coef_
        intercept = model.fit(factorData.iloc[:,0:col], factorData.iloc[:,col]).intercept_
        residual = factorData.iloc[:,col] - (factorData.iloc[:,0:col] @ betaCoef) + intercept
        # Recalculate factor data using residual
        factorData.iloc[:,col] = residual

print("Factor data after: ")
print(factorData.head(5))

Factor data before: 
              ICE_T1  CME_HG10  CME_NN10  EUREX_FMEM2   CME_TN1  CME_GC10  \
Date                                                                        
2019-01-03  0.011063 -0.020904 -0.001595    -0.015792  0.008977  0.007372   
2019-01-04  0.017721  0.028973  0.023749     0.026423 -0.009392 -0.007281   
2019-01-07  0.010922 -0.003697  0.006709     0.001308 -0.002286  0.002415   
2019-01-08  0.025214  0.009262  0.009527     0.000865 -0.003010 -0.003794   
2019-01-09  0.051073  0.000421  0.010509     0.021673 -0.001332  0.003913   

            LIFFE_EMA2  
Date                    
2019-01-03    0.004205  
2019-01-04    0.008337  
2019-01-07    0.019242  
2019-01-08    0.000038  
2019-01-09   -0.004000  
Factor data after: 
              ICE_T1  CME_HG10  CME_NN10  EUREX_FMEM2   CME_TN1  CME_GC10  \
Date                                                                        
2019-01-03  0.011063 -0.021932 -0.000824    -0.006610  0.006348  0.005286   
2019-01-04  0